In [ ]:

# Importuojame reikiamas bibliotekas
import pandas as pd
import pyodbc
import sqlalchemy

In [ ]:
# Nuskaitome dviračių pardavimo duomenis iš CSV failo
df = pd.read_csv("bike_sales_100k.csv")

In [ ]:
# Konvertuojame datos stulpelį į datetime formatą
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')
# Konvertuojame skaitinius stulpelius
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Customer_Age'] = pd.to_numeric(df['Customer_Age'], errors='coerce')

In [ ]:
# Peržiūrime pirmas 5 duomenų eilutes
display(df.head())

,Sale_ID,Date,Customer_ID,Bike_Model,Price,Quantity,Store_Location,Salesperson_ID,Payment_Method,Customer_Age,Customer_Gender
0,1,2022-07-11,9390,Cruiser,318.32,1,Philadelphia,589,Apple Pay,70,Female
1,2,2024-05-03,3374,Hybrid Bike,3093.47,4,Chicago,390,Apple Pay,37,Male
2,3,2022-09-01,2689,Folding Bike,4247.99,3,San Antonio,338,PayPal,59,Female
3,4,2022-09-28,3797,Mountain Bike,1722.01,3,San Antonio,352,Apple Pay,19,Male
4,5,2021-01-05,1633,BMX,3941.44,3,Philadelphia,580,PayPal,67,Female


In [ ]:
# Pakeičiame NaN reikšmes į None (SQL NULL reikšmėms)
df = df.where(pd.notnull(df), None)

In [ ]:
# SQL Server prisijungimo parametrai
conn_str = (
    r"Driver={ODBC Driver 17 for SQL Server};"
    r"Server=USERNAME;"
    r"Database=BikeStoreDB;"
    r"Trusted_Connection=yes;"
)


In [ ]:
# Sukuriame prisijungimą prie duomenų bazės
conn = pyodbc.connect(conn_str, autocommit=False)
cursor = conn.cursor()
cursor.fast_executemany = True

In [ ]:
# SQL užklausa duomenų įterpimui
insert_sql = """
INSERT INTO BikeSales
(Sale_ID, Date, Customer_ID, Bike_Model, Price, Quantity, Store_Location, Salesperson_ID, Payment_Method, Customer_Age, Customer_Gender)
VALUES (?,?,?,?,?,?,?,?,?,?,?)
"""

In [ ]:
# Stulpelių sąrašas duomenų įkėlimui
cols = ['Sale_ID','Date','Customer_ID','Bike_Model','Price','Quantity','Store_Location','Salesperson_ID','Payment_Method','Customer_Age','Customer_Gender']
data_iterable = (tuple(x) for x in df[cols].itertuples(index=False, name=None))

In [ ]:
# Paruošiame duomenis įkėlimui dalimis (chunks)
chunk_size = 5000 # Įkeliame po 5000 įrašų vienu kartu
data_list = list(df[cols].itertuples(index=False, name=None))
# Įkeliame duomenis dalimis, kad išvengtume atminties problemų
for i in range(0, len(data_list), chunk_size):
    chunk = data_list[i:i+chunk_size]
    cursor.executemany(insert_sql, chunk)
    conn.commit() # Išsaugome pakeitimus duomenų bazėje
    
    print(f"Inserted rows {i}..{i+len(chunk)-1}")


Inserted rows 0..4999
Inserted rows 5000..9999
Inserted rows 10000..14999
Inserted rows 15000..19999
Inserted rows 20000..24999
Inserted rows 25000..29999
Inserted rows 30000..34999
Inserted rows 35000..39999
Inserted rows 40000..44999
Inserted rows 45000..49999
Inserted rows 50000..54999
Inserted rows 55000..59999
Inserted rows 60000..64999
Inserted rows 65000..69999
Inserted rows 70000..74999
Inserted rows 75000..79999
Inserted rows 80000..84999
Inserted rows 85000..89999
Inserted rows 90000..94999
Inserted rows 95000..99999


In [ ]:
# Uždarome kursorių ir prisijungimą
cursor.close()
conn.close()
print("Upload finished.")

Upload finished.
